In [1]:
import requests
import json
import pandas as pd
from sqlalchemy import create_engine
import pprint
pp = pprint.PrettyPrinter(indent=4)
import psycopg2 as pg
import pandas.io.sql as pd_sql
from copy import deepcopy
import numpy as np


In [2]:
# connecting to PostgrSQL to retrieve the data needed to make a full dataset for Tableau manipulations

In [3]:
# Twitch Client ID 
clientID = 'vb2kmh60pt0tee6o2c11ko6n2t1w9a'

In [4]:
# Postgres info to connect

connection_args = {
 "host": "twitchdata.chd4n5ul8muk.us-east-2.rds.amazonaws.com",
 "user": "postgres",
   "password":"FwwBFmleh65qYxKxDVb9",
 "port": 5432,
 "dbname": "twitchdata"
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)

In [5]:
#getting the genres and languages for all games to be able to filter the data

In [6]:
query = '''SELECT game_name, game_genres, language FROM general_view GROUP BY game_name, game_genres, language;
 '''
game_genre_language = pd_sql.read_sql(query, connection)

In [7]:
pwd

'/Users/jchow/Work/MetisCode/twitch-streamer-recommender/gitRoot/Twitch_Streamers_Recommender/Recommender_For_Existing_Streamers'

In [10]:
sully = pd.read_csv('../Jeremy_Notebooks/Natasha.csv', header = 0, names = ['date', 'game_name', 'hourly_channels', 'hourly_viewers'] )

In [30]:
import pickle

In [31]:
genres = pickle.load(open( "./Data/final_games (1).p", "rb"))

In [251]:
# making a dictionary of languages and genres to later attach to the data from sully gnome to be able to see trends filtered

In [35]:
genre_dict = pd.Series(genres.category.values,index=genres.game).to_dict()

In [36]:
sully['genres'] = sully['game_name'].map(genre_dict)

In [53]:
query = '''SELECT game_name, language FROM general_view group by game_name, language order by game_name;
 '''
lang = pd_sql.read_sql(query, connection)

In [68]:
games = set(lang['game_name'].to_list())

In [70]:
lang_dict = {}

for game in games:
    lang_dict[game] = lang[lang['game_name']==game]['language'].to_list()

In [71]:
lang_dict

{'Indiana Jones and the Fate of Atlantis': ['en', 'pt'],
 'Blade and Soul': ['zh', 'ko', 'ru', 'en'],
 'ToeJam & Earl: Back in the Groove!': ['en'],
 'Final Fantasy X': ['de', 'fr', 'en', 'zh'],
 'Vacation Simulator': ['en'],
 'PUBG MOBILE': ['vi',
  'ko',
  'de',
  'el',
  'zh-hk',
  'other',
  'es',
  'ro',
  'cs',
  'tr',
  'th',
  'id',
  'en',
  'sv',
  'uk',
  'ar',
  'nl',
  'zh',
  'it',
  'ru',
  'pt',
  'hu',
  'ja',
  'fr',
  'tl',
  'hi'],
 'Battlefield V': ['cs',
  'en',
  'bg',
  'zh',
  'ko',
  'other',
  'pl',
  'tr',
  'ru',
  'sk',
  'es',
  'it',
  'id',
  'el',
  'sv',
  'fi',
  'nl',
  'ro',
  'ar',
  'fr',
  'da',
  'pt',
  'no',
  'ja',
  'th',
  'hu',
  'de'],
 'Project Zomboid': ['tr', 'en'],
 'Call of Duty: WWII': ['en', 'fr', 'pt'],
 'Street Boyz': ['en'],
 'Contradiction: Spot The Liar!': ['en'],
 'Stellaris': ['hu',
  'sv',
  'de',
  'it',
  'ja',
  'es',
  'fr',
  'pt',
  'tr',
  'ru',
  'ko',
  'pl',
  'other',
  'zh',
  'ro',
  'en',
  'cs',
  'nl'],
 'Y

In [72]:
sully['languages'] = sully['game_name'].map(lang_dict)

In [74]:
sully = sully.dropna()

In [127]:
sully['genres'] = sully['genres'].apply(eval)

In [128]:
sully['languages'] = sully['languages'].apply(eval)

In [129]:
sully = sully.genres.apply(pd.Series) \
    .merge(sully, left_index = True, right_index = True) \
    .drop(["genres"], axis = 1)\
    .melt(id_vars = ['game_name', 'date', 'hourly_channels', 'hourly_viewers', 'languages'], value_name = "genres") \
    .drop("variable", axis = 1) \
    .dropna()

In [133]:
sully = sully.languages.apply(pd.Series) \
    .merge(sully, left_index = True, right_index = True) \
    .drop(["languages"], axis = 1)\
    .melt(id_vars = ['game_name', 'date', 'hourly_channels', 'hourly_viewers', 'genres'], value_name = "languages") \
    .drop("variable", axis = 1) \
    .dropna()

In [146]:
sully['channel_change'] = sully['hourly_channels'].diff()

In [147]:
sully['viewership_change'] = sully['hourly_viewers'].diff()

In [149]:
sully = sully.dropna()

In [216]:
sully_sorted = sully.sort_values(['game_name', 'genres', 'languages', 'date'], ascending=[True, True, True, True])


In [241]:
sully_sorted['channel_diff'] = sully_sorted.hourly_channels.diff(periods = 24)

In [245]:
sully_sorted['viewers_diff'] = sully_sorted.hourly_viewers.diff(periods = 24)

In [248]:
#sully_sorted.to_csv('sully_sorted.csv', index = False)

In [ ]:
sully_sorted = pd.read_csv('./Data/sully_sorted.csv')

In [6]:
sully_sorted['channel_diff_percent'] = sully_sorted.hourly_channels.pct_change(periods=24)

In [7]:
sully_sorted['viewers_diff_percent'] = sully_sorted.hourly_viewers.pct_change(periods=24)

In [9]:
sully_sorted['ratio_of_diff'] = sully_sorted.viewers_diff_percent/sully_sorted.channel_diff_percent

In [10]:
sully_sorted

,game_name,date,hourly_channels,hourly_viewers,genres,languages,channel_change,viewership_change,channel_diff,viewers_diff,channel_diff_percent,viewers_diff_percent,ratio_of_diff
0,7 Days to Die,2019-04-22 00:00:00.000,83,506,FPS,de,63.0,302.0,NaN,NaN,NaN,NaN,NaN
1,7 Days to Die,2019-04-22 01:00:00.000,79,421,FPS,de,-4.0,-85.0,NaN,NaN,NaN,NaN,NaN
2,7 Days to Die,2019-04-22 02:00:00.000,73,397,FPS,de,-6.0,-24.0,NaN,NaN,NaN,NaN,NaN
3,7 Days to Die,2019-04-22 03:00:00.000,63,346,FPS,de,-10.0,-51.0,NaN,NaN,NaN,NaN,NaN
4,7 Days to Die,2019-04-22 04:00:00.000,55,345,FPS,de,-8.0,-1.0,NaN,NaN,NaN,NaN,NaN
5,7 Days to Die,2019-04-22 05:00:00.000,52,364,FPS,de,-3.0,19.0,NaN,NaN,NaN,NaN,NaN
6,7 Days to Die,2019-04-22 06:00:00.000,42,299,FPS,de,-10.0,-65.0,NaN,NaN,NaN,NaN,NaN
7,7 Days to Die,2019-04-22 07:00:00.000,34,238,FPS,de,-8.0,-61.0,NaN,NaN,NaN,NaN,NaN
8,7 Days to Die,2019-04-22 08:00:00.000,26,215,FPS,de,-8.0,-23.0,NaN,NaN,NaN,NaN,NaN
9,7 Days to Die,2019-04-22 09:00:00.000,29,195,FPS,de,3.0,-20.0,NaN,NaN,NaN,NaN,NaN


In [11]:
sully_sorted.to_csv('sully_sorted.csv', index = False)

In [14]:
sully_sorted['view_ratio'] = (sully_sorted.hourly_viewers/sully_sorted.hourly_viewers)/(sully_sorted.hourly_viewers.diff(periods = 24)/sully_sorted.hourly_channels.diff(periods = 24))

In [15]:
sully_sorted.head(30)

,game_name,date,hourly_channels,hourly_viewers,genres,languages,channel_change,viewership_change,channel_diff,viewers_diff,channel_diff_percent,viewers_diff_percent,ratio_of_diff,view_ratio
0,7 Days to Die,2019-04-22 00:00:00.000,83,506,FPS,de,63.0,302.0,NaN,NaN,NaN,NaN,NaN,NaN
1,7 Days to Die,2019-04-22 01:00:00.000,79,421,FPS,de,-4.0,-85.0,NaN,NaN,NaN,NaN,NaN,NaN
2,7 Days to Die,2019-04-22 02:00:00.000,73,397,FPS,de,-6.0,-24.0,NaN,NaN,NaN,NaN,NaN,NaN
3,7 Days to Die,2019-04-22 03:00:00.000,63,346,FPS,de,-10.0,-51.0,NaN,NaN,NaN,NaN,NaN,NaN
4,7 Days to Die,2019-04-22 04:00:00.000,55,345,FPS,de,-8.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,7 Days to Die,2019-04-22 05:00:00.000,52,364,FPS,de,-3.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN
6,7 Days to Die,2019-04-22 06:00:00.000,42,299,FPS,de,-10.0,-65.0,NaN,NaN,NaN,NaN,NaN,NaN
7,7 Days to Die,2019-04-22 07:00:00.000,34,238,FPS,de,-8.0,-61.0,NaN,NaN,NaN,NaN,NaN,NaN
8,7 Days to Die,2019-04-22 08:00:00.000,26,215,FPS,de,-8.0,-23.0,NaN,NaN,NaN,NaN,NaN,NaN
9,7 Days to Die,2019-04-22 09:00:00.000,29,195,FPS,de,3.0,-20.0,NaN,NaN,NaN,NaN,NaN,NaN
